<h2>Installing Ollama</h2>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [48]:
# Thanks to:
# https://www.kaggle.com/code/adriensales/ollama-running-local-models-w-llamaindex-cpu
import subprocess
import time
# Start ollama as a backrgound process
command = "env OLLAMA_NUM_PARALLEL=3 nohup ollama serve&"
# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Process ID:", process.pid)
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 948


In [ ]:
!ollama pull phi3

In [4]:
!pip install -q datasets dspy-ai >/dev/null 2>&1

<h1>Get dataset</h1> Corpus of Linguistic Acceptability (CoLA) consists of sentences from 23 linguistics publications, expertly annotated for acceptability (grammaticality) 

In [5]:
from datasets import load_dataset

dataset = load_dataset("linxinyuan/cola")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

<h2>Dspy - Prepare data</h2>

In [6]:
import dspy
from dspy.teleprompt import COPRO
train_ds=dataset['train'].shuffle()[:50]
sentences,labels=train_ds['text'],train_ds['label']
trainset= [dspy.Example(sentence=sentence, answer=str(label)).with_inputs("sentence") for sentence,label in zip(sentences,labels)]
test_ds=dataset['test'].shuffle()[:50]
sentences_test,labels_test=test_ds['text'],test_ds['label']
testset= [dspy.Example(sentence=sentence, answer=str(label)).with_inputs("sentence") for sentence,label in zip(sentences_test,labels_test)]


In [9]:
testset[:5]

[Example({'sentence': 'This proved a decisive factor.', 'answer': '1'}) (input_keys={'sentence'}),
 Example({'sentence': 'he had spent five thousand dollars.', 'answer': '1'}) (input_keys={'sentence'}),
 Example({'sentence': 'Nora sent Peter the book.', 'answer': '1'}) (input_keys={'sentence'}),
 Example({'sentence': 'Henry found that Bill is sad.', 'answer': '1'}) (input_keys={'sentence'}),
 Example({'sentence': 'I found the place where we can relax.', 'answer': '1'}) (input_keys={'sentence'})]

<h2>Dspy - langauge model</h2>

In [10]:
lm = dspy.OllamaLocal(model='phi3',timeout_s=300)
dspy.configure(lm=lm)

<h2>Dspy - Signature(unrefined) : our initial unrefined prompt. </h2>
    <p>The idea is not to get the best results for classification, but to see if dspy will be able to give us a somewhat refined version.</p>

In [11]:
class GetLabel(dspy.Signature):
    """Check if sentence is correct(1) or not(0)"""

    sentence = dspy.InputField(
        prefix = "Sentence:",
        desc = "a sentence in English",
    )
    answer = dspy.OutputField(
        prefix ="Predicted label:",
        desc="predicted label only (1 or 0 only)"
    )

class AnswerBot(dspy.Module):
    def __init__(self, tools=None):
        super().__init__()

        self.generate_answer = dspy.Predict(GetLabel)

    def forward(self, sentence):
        prediction = self.generate_answer(sentence=f"{sentence}")
        return dspy.Prediction(answer=prediction.answer)

uncompiled_answer_bot = AnswerBot()


In [12]:
def validate_answer(example, pred, trace=None):

    answer_match = example.answer.lower().strip() == pred.answer.lower().strip()

    return answer_match

In [13]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=50)



<h2>Evaluate uncompiled</h2>

In [17]:
# Launch evaluation.
evaluator(uncompiled_answer_bot, metric=validate_answer)

Average Metric: 4 / 50  (8.0): 100%|██████████| 50/50 [01:01<00:00,  1.22s/it] 


,sentence,example_answer,pred_answer,validate_answer
0,This proved a decisive factor.,1,"Predicted label: 1 The sentence ""This proved a decisive factor."" is grammatically correct and conveys a clear meaning, so the predicted label is 1 (correct).",False
1,he had spent five thousand dollars.,1,"Predicted label: 1 The sentence ""he had spent five thousand dollars."" is grammatically correct and makes sense, so the predicted label is 1.",False
2,Nora sent Peter the book.,1,"Predicted label: 1 The sentence ""Nora sent Peter the book."" is grammatically correct, with a clear subject (Nora), verb (sent), and object (the book). The...",False
3,Henry found that Bill is sad.,1,"Predicted label: 1 The sentence ""Henry found that Bill is sad."" is grammatically correct, with a clear subject (""Henry""), verb (""found""), and complementary clause expressing...",False
4,I found the place where we can relax.,1,"Predicted label: 1 The sentence ""I found the place where we can relax."" is grammatically correct and makes sense, so it receives a predicted label...",False
5,"I know which book Mag read, and which book Bob said that you hadn't.",1,"Predicted label: 0 The sentence ""I know which book Mag read, and which book Bob said that you hadn't."" contains a grammatical error with the...",False
6,Kim alienated cats and beating his dog.,0,"Predicted label: 0 The sentence ""Kim alienated cats and beating his dog."" is not correct due to grammatical errors, specifically the lack of a verb...",False
7,"Most people probably consider, even though the courts didn't actually find, Klaus guilty of murder.",1,1 The given sentence is grammatically correct and conveys a clear meaning. It discusses an assumption about public opinion regarding a court decision involving a...,False
8,Came right in he did without so much as a knock.,1,"1 The sentence ""Came right in he did without so much as a knock."" is grammatically correct, although it may sound slightly informal or conversational....",False
9,We pulled free.,1,Sentence: We pulled free. Predicted label: 1,False


8.0

<h4>The responses are not bad but the score is. We just need label without any explanation and without any extra words/labels</h4>

In [18]:
lm.history[-1]['prompt']

'Check if sentence is correct(1) or not(0)\n\n---\n\nFollow the following format.\n\nSentence: a sentence in English\nPredicted label: predicted label only (1 or 0 only)\n\n---\n\nSentence: I know you eat asparagus.\nPredicted label:'

<h2>Let's see if we can get a better instruction</h2>

In [ ]:
teleprompter = COPRO(metric=validate_answer,depth=2)
kwargs = dict(num_threads=64, display_progress=True) 
compiled_answer_bot = teleprompter.compile(uncompiled_answer_bot, trainset=trainset,eval_kwargs=kwargs)

In [ ]:
compiled_answer_bot.candidate_programs

<h2>Define better signature</h2> by trying the best instructions from the above proposed instructions</h2>

In [14]:
class GetLabelOptimized(dspy.Signature):
    """Determine if the given sentence adheres to standard English grammar rules by outputting a '1' for grammatically correct and '0' for incorrect."""

    sentence = dspy.InputField(
        prefix = "Sentence:",
        desc = "a sentence in English",
    )
    answer = dspy.OutputField(
        prefix="""If the input sentence is grammatically correct, please respond with 1; otherwise, respond with 0.""",
        desc="predicted label only (1 or 0 only)"
    )

class AnswerBotOptimized(dspy.Module):
    def __init__(self, tools=None):
        super().__init__()

        self.generate_answer = dspy.Predict(GetLabelOptimized)

    def forward(self, sentence):
        prediction = self.generate_answer(sentence=f"{sentence}")
        return dspy.Prediction(answer=prediction.answer)

In [15]:
optimized_answer_bot = AnswerBotOptimized()

<h2>Let's test</h2>

In [19]:
# Launch evaluation.
evaluator(optimized_answer_bot, metric=validate_answer)

Average Metric: 42 / 50  (84.0): 100%|██████████| 50/50 [00:12<00:00,  3.99it/s]


,sentence,example_answer,pred_answer,validate_answer
0,This proved a decisive factor.,1,1,✔️ [True]
1,he had spent five thousand dollars.,1,1,✔️ [True]
2,Nora sent Peter the book.,1,1,✔️ [True]
3,Henry found that Bill is sad.,1,1,✔️ [True]
4,I found the place where we can relax.,1,1,✔️ [True]
5,"I know which book Mag read, and which book Bob said that you hadn't.",1,0,False
6,Kim alienated cats and beating his dog.,0,0,✔️ [True]
7,"Most people probably consider, even though the courts didn't actually find, Klaus guilty of murder.",1,1,✔️ [True]
8,Came right in he did without so much as a knock.,1,0,False
9,We pulled free.,1,1,✔️ [True]


84.0

In [21]:
lm.history[-1]['prompt']

"Determine if the given sentence adheres to standard English grammar rules by outputting a '1' for grammatically correct and '0' for incorrect.\n\n---\n\nFollow the following format.\n\nSentence: a sentence in English\nIf the input sentence is grammatically correct, please respond with 1; otherwise, respond with 0. predicted label only (1 or 0 only)\n\n---\n\nSentence: I know you eat asparagus.\nIf the input sentence is grammatically correct, please respond with 1; otherwise, respond with 0."

<h3>much better!</h3>

In [37]:
lm.history[-1]['prompt']

"Determine if the given sentence adheres to standard English grammar rules by outputting a '1' for grammatically correct and '0' for incorrect.\n\n---\n\nFollow the following format.\n\nSentence: a sentence in English\nIf the input sentence is grammatically correct, please respond with 1; otherwise, respond with 0. predicted label only (1 or 0 only)\n\n---\n\nSentence: I know you eat asparagus.\nIf the input sentence is grammatically correct, please respond with 1; otherwise, respond with 0."

In [42]:
dataset["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 527
})